# MNIST with TensorFlow

Code Created by Luis Enrique Acevedo Galicia

Date: 2019-13-02


Here, I present a simple and easy way to create a typlical MNIST classification with TensorFlow.

# The Libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# The model

In [6]:
#model size
size_input = 784
size_output = 10

#hidden layers size (the same for all layers)
size_hl = 50

#clear memory
tf.reset_default_graph()

#define inputs and targets
TF_inputs = tf.placeholder(tf.float32, [None, size_input])
TF_targets = tf.placeholder(tf.float32, [None, size_output])

#----------Layer 1---------

#initialization of weights and bias of layer 1
TF_w1 = tf.get_variable("TF_w1", [size_input, size_hl])
TF_b1 = tf.get_variable("TF_b1", [size_hl])

#set the outputs of layer 1 (ReLu as activation function)
TF_o1 = tf.nn.relu(tf.matmul(TF_inputs, TF_w1) + TF_b1)

#----------Layer 2---------

#initialization of weights and bias of layer 1
TF_w2 = tf.get_variable("TF_w2", [size_hl, size_hl])
TF_b2 = tf.get_variable("TF_b2", [size_hl])

#set the outputs of layer 1 (ReLu as activation function)
TF_o2 = tf.nn.relu(tf.matmul(TF_o1, TF_w2) + TF_b2)

#----------Layer 3---------

#initialization of weights and bias of layer 1
TF_w3 = tf.get_variable("TF_w3", [size_hl, size_output])
TF_b3 = tf.get_variable("TF_b3", [size_output])

#set the outputs of layer 1 (ReLu as activation function)
TF_out = tf.matmul(TF_o2, TF_w3) + TF_b3


#loss function
TF_lf = tf.nn.softmax_cross_entropy_with_logits(logits=TF_out, labels=TF_targets)
#average loss
AV_loss = tf.reduce_mean(TF_lf)

#optimization
TF_opt = tf.train.AdamOptimizer(learning_rate=0.001).minimize(AV_loss)

# Get a 0 or 1 for every input
TF_zero_one = tf.equal(tf.argmax(TF_out, 1), tf.argmax(TF_targets, 1))

# Get the average accuracy of the outputs.
TF_accuracy = tf.reduce_mean(tf.cast(TF_zero_one, tf.float32))

#Prepare execution

TF_sess = tf.InteractiveSession()
TF_ini = tf.global_variables_initializer()
TF_sess.run(TF_ini)

#The batch
size_batch = 100
N_batch =  mnist_data.train._num_examples // size_batch

#early stop
max_epochs = 15
prev_validation_loss = 9999999.

#Epoch loop
for epoch_count in range(max_epochs):
    #batch losses tracking
    curr_epoch_loss = 0.
    
    #batch iteration in epoch
    for batch_counter in range(N_batch):
        
        #assigned input and target batch
        input_batch, target_batch = mnist_data.train.next_batch(size_batch)
       
        # Run the optimization
        _, batch_loss = TF_sess.run([TF_opt, AV_loss], 
            feed_dict={TF_inputs: input_batch, TF_targets: target_batch})
        
        # number of batch losses.
        curr_epoch_loss += batch_loss

    curr_epoch_loss /= N_batch
    
    #Inputs and targets from validation
    input_batch, target_batch = mnist_data.validation.next_batch(mnist_data.validation._num_examples)
    
    #validating
    validation_loss, validation_accuracy = TF_sess.run([AV_loss, TF_accuracy], 
        feed_dict={TF_inputs: input_batch, TF_targets: target_batch})
    
    # Print statistics for the current epoch
    print('Epoch '+str(epoch_count+1)+
          '. Mean loss: '+'{0:.3f}'.format(curr_epoch_loss)+
          '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
          '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
    
    # Trigger early stopping if validation loss begins increasing.
    if validation_loss > prev_validation_loss:
        break
        
    # Store this epoch's validation loss to be used as previous validation loss in the next iteration.
    prev_validation_loss = validation_loss

print('End of training.')



Epoch 1. Mean loss: 0.398. Validation loss: 0.190. Validation accuracy: 94.60%
Epoch 2. Mean loss: 0.174. Validation loss: 0.140. Validation accuracy: 96.22%
Epoch 3. Mean loss: 0.128. Validation loss: 0.112. Validation accuracy: 96.84%
Epoch 4. Mean loss: 0.104. Validation loss: 0.104. Validation accuracy: 96.84%
Epoch 5. Mean loss: 0.087. Validation loss: 0.097. Validation accuracy: 97.12%
Epoch 6. Mean loss: 0.074. Validation loss: 0.097. Validation accuracy: 97.32%
End of training.


# Testing the model

In [9]:
input_batch, target_batch = mnist_data.test.next_batch(mnist_data.test._num_examples)
test_accuracy = TF_sess.run([TF_accuracy], 
    feed_dict={TF_inputs: input_batch, TF_targets: target_batch})


test_accuracy_percent = test_accuracy[0] * 100.

# Print the test accuracy formatted in percentages
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy: 97.17%


In [8]:
test_accuracy_percent

97.17000126838684